<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [246]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler

In [247]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [248]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [249]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [250]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [251]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TRAIN_2 = '../input/parsingauto202109/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [252]:
!ls '../input'

In [253]:
train = pd.concat([pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv'),
                   pd.read_csv(DIR_TRAIN_2+'auto_data_2021.csv')]) # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [254]:
train.sample(5)

In [255]:
train.info()

In [256]:
test.head(5)

In [257]:
test.info()

## Data Preprocessing

In [258]:
train = train[train['price']!=-1]

In [259]:
train.dropna(subset=['productionDate','mileage'], inplace=True)
train.dropna(subset=['price'], inplace=True)

In [260]:
# для baseline просто возьму пару схожих признаков без полной обработки
columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement']
  
df_train = train[columns]
df_test = test[columns]

In [261]:
y = train['price'][train['price']!=-1]

## Label Encoding

In [262]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [263]:
for colum in ['bodyType', 'brand', 'engineDisplacement']:
    data[colum] = data[colum].astype('category').cat.codes

In [264]:
X = data[data['sample'] == 1].drop(['sample'], axis=1)
X_sub = data[data['sample'] == 0].drop(['sample'], axis=1)

## Train Split

In [265]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [266]:
tmp_train = X_train.copy().reset_index(drop=True)
tmp_train['price'] = y_train.reset_index(drop=True)

In [267]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [268]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [269]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [270]:
np.log(y_train)

In [271]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [272]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [273]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Как видим точность возросла до 15%, а что будет на ЛБ?

В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing

## Попробуем обработать наши признаки ##
### Для этого совместим train и test ###

In [274]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест

data = test.append(train, sort=False).reset_index(drop=True) # объединяем

## Очистим данные от излишней информации ##

In [275]:
data.drop(labels=['bodyType','complectation_dict','equipment_dict',
                  'image','model_info','model_name', 'parsing_unixtime',
                 'priceCurrency','sell_id','super_gen','vendor',
                 'Владение','Состояние','Таможня','Комплектация',
                 'start_date', 'hidden', 'model', 'Unnamed: 0', 
                  'vehicleEngine','offers', 'год выпуска', 'Кузов',
                  'Цвет', 'Двигатель', 'Коробка', 'engine_volume', 'key', 'car_url'], axis=1, inplace=True)

data.info()

## Обработаем все признаки в датасете ##

In [276]:
data.info()

Признак - brand. С брэндами авто мы ничего не будем делать, просто отнесем к каждому брэнду числовое значения

In [277]:
data['brand'].isna().sum()

Признак - color. Приведем к одному формату все значения в столбце и каждый цвет отнесем числовое значение

In [278]:
data['color'].value_counts()

In [279]:
colors = {'040001': 'черный', 'чёрный': 'черный', 'FAFBFB': 'серебристый',
          'белый': 'белый', 'CACECB': 'серый', '97948F': 'серый', '0000CC': 'синий',
       'серый': 'серый', 'серебристый': 'серебристый', 'синий': 'синий', 'EE1D19': 'красный', '200204': 'бордо',
          'красный': 'красный','коричневый': 'коричневый', '007F00': 'зеленый', 'C49648': 'коричневый', 
          'зелёный': 'зелёный', '22A0F8': 'голубой', 'бежевый': 'бежевый', 'голубой': 'голубой',
          'DEA522': 'желтый', '660099': 'пурпурный', 'FFD600': 'желтый', '4A2197': 'синий', 'FF8649': 'оранжевый',
       'золотистый': 'золотистый', 'пурпурный': 'пурпурный', 'фиолетовый': 'фиолетовый', 'жёлтый': 'жёлтый',
          'оранжевый': 'оранжевый', 'розовый': 'розовый', 'FFC0CB': 'розовый'}


data['color'] = data['color'].map(colors)

Признак - description  с описанием к объявлению. Будем использовать, чтобы создать признаки: длина описания и различие дилеров и физлиц

In [280]:
data['desc_len'] = data['description'].apply(lambda x: len(str(x)))
data['dealers'] = data['description'].map(dict(data['description'].value_counts()))

Признак - engineDisplacement. Значений много, поэтому просто присвоим числовые значения

In [281]:
data['engineDisplacement'].value_counts()

Признак - enginePower. Значений много, поэтому просто присвоим числовые значения

In [282]:
data['enginePower'].value_counts()

Признак - fuelType

In [283]:
data['fuelType'].value_counts()

Признак - mileage

In [284]:
data['mileage'].value_counts()

Признак - modelDate

In [285]:
data['modelDate'].value_counts()

Признак - name. Удалим данный признак - вся информация есть в других признаках

In [286]:
data['name'].value_counts()

Признак - numberOfDoors

In [287]:
data['numberOfDoors'].value_counts()

Признак - productionDate

In [288]:
data['productionDate'].value_counts()

Признак - vehicleConfiguration

In [289]:
data['vehicleConfiguration'].value_counts()

Признак - vehicleTransmission

In [290]:
data['vehicleTransmission'].value_counts()

In [291]:
def change_trans(x):
    if x == 'AUTOMATIC':
        return 'автоматическая'
    elif x == 'MECHANICAL':
        return 'механическая'
    elif x == 'ROBOT':
        return 'роботизированная'
    elif x == 'VARIATOR':
        return 'вариатор'
    
data['vehicleTransmission'] = data['vehicleTransmission'].apply(change_trans)

In [292]:
data['vehicleTransmission'].value_counts()

Признак - Владельцы

In [293]:
data['Владельцы'].value_counts()

In [294]:
def change_owners(x):
    if x == '3 и или более' or x == 3.0:
        return 3.0
    elif x == '1 владелец' or x == 1.0:
        return 1.0
    elif x == '2 владельца' or x == 2.0:
        return 2.0
    
data['Владельцы'] = data['Владельцы'].apply(change_owners)

In [295]:
data['Владельцы'].value_counts()

Признак - ПТС

In [296]:
data['ПТС'].value_counts()

In [297]:
def change_pts(x):
    if x == 'ORIGINAL' or x == 'Оригинал':
        return 'Оригинал'
    else:
        return 'Дубликат'
    
data['ПТС'] = data['ПТС'].apply(change_pts)

In [298]:
data['ПТС'].value_counts()

Признак - Привод

In [299]:
data['Привод'].value_counts()

Признак - Руль     

In [300]:
data['Руль'].value_counts()

In [301]:
def change_wheel(x):
    if x == 'LEFT' or x == 'Левый':
        return 'левый'
    else:
        return 'правый'
    
data['Руль'] = data['Руль'].apply(change_wheel)

In [302]:
data

In [303]:
#список признаков для модели
columns = ['brand','color','engineDisplacement','enginePower','fuelType','vehicleConfiguration','vehicleTransmission',
          'Владельцы','ПТС', 'Привод','Руль']

for colum in columns:
    data[colum] = data[colum].astype('category').cat.codes

In [304]:
data.drop(labels=['description','name'], axis=1, inplace=True)

In [305]:
data['mileage'].fillna(data['mileage'].median(), inplace=True)

In [306]:
data['modelDate'].fillna(data['modelDate'].median(), inplace=True)

In [307]:
data['numberOfDoors'].fillna(data['numberOfDoors'].median(), inplace=True)

In [308]:
data['dealers'].fillna(data['dealers'].value_counts().index[0], inplace=True)

Попробуем удалить enginePower, потому что у этого признака высокая корреляция с двумя другими признаками (vehicleTransmission - (-0.77) и Владельцы - (-0.71) - оставим этот признак, потому что результат немного ухудшился

In [309]:
#data.drop(labels=['enginePower'], axis=1, inplace=True)

In [310]:
import matplotlib.pyplot as plt
import seaborn as sns

In [311]:
plt.rcParams['figure.figsize'] = (12,12)
sns.heatmap(data.drop(['sample'], axis=1).corr(),annot=True, fmt=".2f", linewidths=0.1, cbar=False)

Подготовим данные

In [312]:
num_cols = ['engineDisplacement','mileage','modelDate', 'enginePower', #
            'productionDate','desc_len','dealers']


# Стандартизация числовых переменных
for column in num_cols:
    scaler = StandardScaler()
    data.loc[:, [column]] = scaler.fit_transform(data.loc[:,[column]]) 

In [313]:
data.info()

## Теперь построим модель на cat_boost, как ранее.
Я вручную менял параметры модели и лучший результат у меня получился - 12.68%, а в ЛБ - 16.3909

В результате исключения признака enginePower получился результат - 13.15%, а в ЛБ - 16.39991 (изменения небольшие в худшую сторону, оставим этот признак)

In [314]:
y = data['price'][(data['price']!=-1) & (data['sample']==1)]

In [315]:
X = data[data['sample'] == 1].drop(['sample','price'], axis=1)
X_sub = data[data['sample'] == 0].drop(['sample','price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [316]:
y

In [317]:
model = CatBoostRegressor(iterations = 5000,
                          learning_rate=0.1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

In [318]:
model.get_best_score(), model.get_best_iteration()

In [319]:
model = CatBoostRegressor(iterations = 5000,
                          learning_rate=0.1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

In [320]:
# есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')

In [321]:
model.get_best_score(), model.get_best_iteration()

In [322]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [323]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# Submission

Так как X_sub данные были собраны в 2020 году, а я собирал свои данные в 2021 году. Год выдался очень необычный из-за пандемии, которая повлияла на все сферы жизни включая рынок машин, я нашел коэффицент на который умножал прогноз модели результат значительно улучшается. Его значение равно 0.87

In [324]:
sample_submission['price'] = predict_submission * 0.87
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)